# otsu

In [ ]:
import numpy as np
import pandas as pd
from skimage.filters import threshold_otsu

def otsu_thresholding(df, marker_columns):
    """
    Apply Otsu's thresholding method to each marker column in the dataset using skimage library.

    Args:
    df (pd.DataFrame): DataFrame containing marker intensity values.
    marker_columns (list): List of marker columns to compute the thresholds for.

    Returns:
    dict: A dictionary with the optimal thresholds for each marker.
    """
    thresholds = {}

    for marker in marker_columns:
        # Get the intensity values for the current marker
        intensities = df[marker].values

        # Use skimage's Otsu method to find the optimal threshold
        optimal_threshold = threshold_otsu(intensities)

        # Store the optimal threshold for the current marker
        thresholds[marker] = optimal_threshold

    return thresholds

# Example usage:
# df = pd.read_csv("your_data.csv")
# marker_columns = ['CD66b', 'CD56', 'CD4', 'CTLA4', 'CD8', 'CD20']
# thresholds = otsu_thresholding(df, marker_columns)
# print(thresholds)


# IsoData

In [ ]:
import numpy as np
import pandas as pd

def isodata_thresholding(df, marker_columns, epsilon=1e-5, max_iter=1000):
    """
    Apply the IsoData thresholding method to each marker column in the dataset.

    Args:
    df (pd.DataFrame): DataFrame containing marker intensity values.
    marker_columns (list): List of marker columns to compute the thresholds for.
    epsilon (float): Convergence tolerance for the threshold update.
    max_iter (int): Maximum number of iterations allowed.

    Returns:
    dict: A dictionary with the optimal thresholds for each marker.
    """
    thresholds = {}

    for marker in marker_columns:
        # Get the intensity values for the current marker
        intensities = df[marker].values

        # Initialize the threshold as the mean of the intensities
        threshold = np.mean(intensities)

        for _ in range(max_iter):
            # Partition the cells into two classes based on the current threshold
            C0 = intensities[intensities <= threshold]
            C1 = intensities[intensities > threshold]

            if len(C0) == 0 or len(C1) == 0:
                break

            # Compute the means of the two classes
            mu_0 = np.mean(C0)
            mu_1 = np.mean(C1)

            # Update the threshold as the average of the two class means
            new_threshold = (mu_0 + mu_1) / 2

            # Check for convergence
            if np.abs(new_threshold - threshold) < epsilon:
                break

            # Update the threshold for the next iteration
            threshold = new_threshold

        # Store the final threshold for the current marker
        thresholds[marker] = threshold

    return thresholds

# Example usage:
# df = pd.read_csv("your_data.csv")
# marker_columns = ['CD66b', 'CD56', 'CD4', 'CTLA4', 'CD8', 'CD20']
# thresholds_isodata = isodata_thresholding(df, marker_columns)
# print(thresholds_isodata)


# A Modified Version of GMM

In [ ]:
from sklearn.mixture import GaussianMixture
import numpy as np

def apply_gmm_thresholding(data_df, markers, max_components=10, random_state=42):
    thresholds = {}

    for marker in markers:
        # Extract marker values
        marker_values = data_df[[marker]].values

        # Best GMM selection based on BIC
        best_gmm = None
        lowest_bic = np.inf
        best_n_components = 2  # Start with 2 components by default

        # Try different number of components (K)
        for n_components in range(2, max_components + 1):
            gmm = GaussianMixture(n_components=n_components, random_state=random_state)
            gmm.fit(marker_values)
            bic = gmm.bic(marker_values)

            if bic < lowest_bic:
                lowest_bic = bic
                best_gmm = gmm
                best_n_components = n_components

        # Handle K = 2 case (simple thresholding)
        if best_n_components == 2:
            threshold = np.mean(best_gmm.means_)

        # Handle K > 2 case (custom thresholding based on largest gap)
        else:
            means = np.sort(best_gmm.means_.flatten())
            delta_means = np.diff(means)
            k_boundary = np.argmax(delta_means)
            threshold = (means[k_boundary] + means[k_boundary + 1]) / 2

        # Store threshold and create binary column for above threshold
        thresholds[marker] = threshold


    return thresholds

# Example usage:
# data_df = pd.read_csv("your_data.csv")
# markers = ['CD66b', 'CD56', 'CD4', 'CTLA4', 'CD8', 'CD20']
# updated_df, thresholds = apply_gmm_thresholding(data_df, markers)
# print("GMM Thresholds:", thresholds)


# minimum cross-entropy

In [ ]:
def min_cross_entropy_thresholding(data_df, marker_columns, candidate_thresholds, dist_name='normal'):
    """
    Apply Minimum Cross-Entropy Thresholding for each marker in the DataFrame.

    Args:
    data_df: DataFrame containing the marker intensity values.
    marker_columns: List of marker columns to apply the thresholding.
    candidate_thresholds: Dictionary of candidate thresholds for each marker.
    dist_name: Distribution to use ('normal', 'lognormal', 'gamma', 'exponential').

    Returns:
    dict: A dictionary with the optimal threshold for each marker.
    """
    thresholds = {}

    for marker in marker_columns:
        data = data_df[marker].values
        unique_thresholds = candidate_thresholds[marker]  # Now correctly referencing marker-specific thresholds

        def objective(threshold):
            non_expressing = data[data <= threshold]
            expressing = data[data > threshold]

            if len(non_expressing) == 0 or len(expressing) == 0:
                return np.inf  # Avoid empty class case

            dist_non_exp = fit_distribution(non_expressing, dist_name)
            dist_exp = fit_distribution(expressing, dist_name)

            if dist_non_exp is None or dist_exp is None:
                return np.inf

            # Compute cross-entropy
            p0 = dist_non_exp.pdf(non_expressing)
            p1 = dist_exp.pdf(expressing)

            ce_non_exp = cross_entropy(non_expressing, p0)
            ce_exp = cross_entropy(expressing, p1)

            return ce_non_exp + ce_exp

        # Use scipy's minimize to find the optimal threshold
        result = minimize(objective, x0=[np.median(data)], bounds=[(min(data), max(data))])

        # Store the best threshold
        thresholds[marker] = result.x[0] if result.success else None

    return thresholds


# generate candiadate thresholds

In [ ]:
def generate_candidate_thresholds(df, marker_columns):
    """
    Generate candidate thresholds based on unique intensity values for each marker.

    Args:
    df (pd.DataFrame): DataFrame containing marker intensity values.
    marker_columns (list): List of marker columns to compute the thresholds for.

    Returns:
    dict: A dictionary with marker names as keys and their corresponding unique intensity thresholds as values.
    """
    candidate_thresholds = {}

    for marker in marker_columns:
        # Get the unique intensity values for the current marker, sorted
        unique_intensities = np.sort(df[marker].unique())
        candidate_thresholds[marker] = unique_intensities

    return candidate_thresholds

# Example usage:
# df = pd.read_csv("your_data.csv")
# marker_columns = ['CD66b', 'CD56', 'CD4', 'CTLA4', 'CD8', 'CD20']
# candidate_thresholds = generate_candidate_thresholds(df, marker_columns)
# print(candidate_thresholds)


# load and run data

In [ ]:
 import pandas as pd

# Load your data
df = pd.read_csv("/content/umap_filtered_002_TU2_Immune_2_thresholded_encoded.csv")

marker_columns = ['CD66b', 'CD56', 'CD4', 'CTLA4', 'CD8', 'CD20']

thresholds_otsu = otsu_thresholding(df, marker_columns)
print("Otsu Thresholds:", thresholds_otsu)


thresholds_isodata = isodata_thresholding(df, marker_columns)
print("IsoData Thresholds:", thresholds_isodata)

thresholds_gmm = modified_gmm_thresholding(df, marker_columns)
print("GMM Thresholds:", thresholds_gmm)

candidate_thresholds = generate_candidate_thresholds(df, marker_columns)


for marker in marker_columns:
    thresholds_mce = min_cross_entropy_thresholding(df, [marker], candidate_thresholds)
    print(f"Minimum Cross-Entropy Threshold for {marker}: {thresholds_mce}")



Otsu Thresholds: {'CD66b': 11.399895365332032, 'CD56': 3.4436964739765625, 'CD4': 4.638822374882812, 'CTLA4': 1.7203896906113283, 'CD8': 2.993261530101562, 'CD20': 1.8819881087285157}
IsoData Thresholds: {'CD66b': 11.508348076880885, 'CD56': 3.468594647486471, 'CD4': 4.659877346459998, 'CTLA4': 1.7343134856874514, 'CD8': 3.0127165920619516, 'CD20': 1.885931165886635}


/usr/local/lib/python3.10/dist-packages/sklearn/mixture/_base.py:270: ConvergenceWarning: Best performing initialization did not converge. Try different init parameters, or increase max_iter, tol, or check for degenerate data.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/mixture/_base.py:270: ConvergenceWarning: Best performing initialization did not converge. Try different init parameters, or increase max_iter, tol, or check for degenerate data.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/mixture/_base.py:270: ConvergenceWarning: Best performing initialization did not converge. Try different init parameters, or increase max_iter, tol, or check for degenerate data.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/mixture/_base.py:270: ConvergenceWarning: Best performing initialization did not converge. Try different init parameters, or increase max_iter, tol, or check for degenerate data.
  warnings.warn(
/usr/local/lib/python3.1

GMM Thresholds: {'CD66b': 59.686612166381806, 'CD56': 8.895729362193556, 'CD4': 5.335314741581406, 'CTLA4': 2.6929802382809442, 'CD8': 6.2596334253947905, 'CD20': 1.4140196167568204}
Minimum Cross-Entropy Threshold for CD66b: {'CD66b': 0.013985693}
Minimum Cross-Entropy Threshold for CD56: {'CD56': 3.255864025}
Minimum Cross-Entropy Threshold for CD4: {'CD4': 2.382756981}
Minimum Cross-Entropy Threshold for CTLA4: {'CTLA4': 1.889522099}
Minimum Cross-Entropy Threshold for CD8: {'CD8': 2.7746521}
Minimum Cross-Entropy Threshold for CD20: {'CD20': 1.802490234}
